In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'ch4': 3.6e-06}
band = [9]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
  dv = 0.001
  klin = 2e-21
  molecule = ch4
  ng_adju = [0, 0]
  ng_refs = [2, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.66, 1.66), (1.66, 1.66, 1.66)]
  wgt = [(0.7, 0.7), (0.4, 0.6, 0.75)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.162503                    1 -0.131756
0.000750        2 -0.160957                    2 -0.130703
0.001052        3 -0.157522                    3 -0.128345
0.001476        4 -0.154079                    4 -0.125999
0.002070        5 -0.150686                    5 -0.123554
0.002904        6 -0.147377                    6 -0.121151
0.004074        7 -0.144214                    7 -0.118640
0.005714        8 -0.141258                    8 -0.116137
0.008015        9 -0.138647                    9 -0.113678
0.011243       10 -0.136512                   10 -0.111190
0.015771       11 -0.135048                   11 -0.108820
0.022122       12 -0.134550                   12 -0.106472
0.031031       13 -0.135424                   13 -0.104379
0.043528       14 -0.137027                   14 -0.101616
0.061057       15 -0.125524                   15 -0.088817
0.085645       16 -0.090179                   16 -0.058394
0.120136       17 -0.039554                   17 -0.014931
0.168516       18  0.020051                   18  0.038107
0.236378       19  0.089523                   19  0.102001
0.331549       20  0.170585                   20  0.178577
0.465100       21  0.266752                   21  0.269723
0.652400       22  0.368675                   22  0.364715
0.915100       23  0.379404                   23  0.388794
1.283650       24  0.299570                   24  0.332629
1.800600       25  0.211841                   25  0.252233
2.525700       26  0.131551                   26  0.163474
3.542800       27  0.079039                   27  0.092770
4.969550       28  0.049771                   28  0.047589
6.970850       29  0.033792                   29  0.025486
9.778100       30  0.024598                   30  0.018383
13.715850      31  0.018359                   31  0.017514
19.239350      32  0.013033                   32  0.016183
26.987250      33  0.007779                   33  0.012321
37.855300      34  0.002415                   34  0.006074
53.100050      35 -0.007483                   35 -0.005967
73.887500      36 -0.021046                   36 -0.022224
97.662500      37 -0.032472                   37 -0.035750
121.437500     38 -0.036877                   38 -0.040955
145.212500     39 -0.035597                   39 -0.039749
168.987500     40 -0.035440                   40 -0.039185
192.762500     41 -0.035098                   41 -0.038488
216.537500     42 -0.034402                   42 -0.037710
240.312500     43 -0.033331                   43 -0.036608
264.087500     44 -0.031901                   44 -0.035346
287.862500     45 -0.030132                   45 -0.033837
311.637500     46 -0.028048                   46 -0.031953
335.412500     47 -0.025668                   47 -0.029972
359.187500     48 -0.023011                   48 -0.027548
382.962500     49 -0.020089                   49 -0.024829
406.737500     50 -0.016907                   50 -0.021798
430.512500     51 -0.013477                   51 -0.018467
454.287500     52 -0.009796                   52 -0.014865
478.062500     53 -0.005866                   53 -0.010752
501.837500     54 -0.001676                   54 -0.006471
525.612500     55  0.002794                   55 -0.001614
549.387500     56  0.007568                   56  0.003485
573.162500     57  0.012692                   57  0.009187
596.937500     58  0.018236                   58  0.015307
620.712500     59  0.024277                   59  0.021990
644.487500     60  0.030870                   60  0.029282
668.262500     61  0.037802                   61  0.036914
692.037500     62  0.044198                   62  0.043955
715.812500     63  0.048590                   63  0.048929
739.587500     64  0.050310                   64  0.051494
763.362500     65  0.050468                   65  0.052455
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -19.783117  0.000000 -19.783117 -19.745032  9.030857e-08   
0.000624    2     -19.783135  0.000005 -19.783129 -19.745045  3.645747e-06   
0.000876    3     -19.783142  0.000008 -19.783134 -19.745051  5.095845e-06   
0.001229    4     -19.783151  0.000011 -19.783141 -19.745058  7.180577e-06   
0.001723    5     -19.783165  0.000015 -19.783150 -19.745069  1.017605e-05   
0.002417    6     -19.783184  0.000022 -19.783162 -19.745083  1.448147e-05   
0.003391    7     -19.783210  0.000031 -19.783179 -19.745103  2.066535e-05   
0.004757    8     -19.783247  0.000044 -19.783202 -19.745131  2.955302e-05   
0.006672    9     -19.783298  0.000064 -19.783234 -19.745170  4.232019e-05   
0.009359    10    -19.783369  0.000091 -19.783279 -19.745225  6.064860e-05   
0.013128    11    -19.783469  0.000129 -19.783340 -19.745301  8.696202e-05   
0.018415    12    -19.783609  0.000184 -19.783424 -19.745407  1.246983e-04   
0.025830    13    -19.783804  0.000262 -19.783542 -19.745554  1.788135e-04   
0.036232    14    -19.784081  0.000371 -19.783709 -19.745761  2.562880e-04   
0.050823    15    -19.784471  0.000525 -19.783946 -19.746048  3.678818e-04   
0.071291    16    -19.785010  0.000760 -19.784250 -19.746436  5.411437e-04   
0.100000    17    -19.785713  0.001156 -19.784557 -19.746932  8.381995e-04   
0.140271    18    -19.786585  0.001840 -19.784746 -19.747527  1.361428e-03   
0.196760    19    -19.787613  0.003002 -19.784612 -19.748185  2.274445e-03   
0.275997    20    -19.788713  0.004942 -19.783771 -19.748802  3.849671e-03   
0.387100    21    -19.789648  0.008122 -19.781526 -19.749135  6.533351e-03   
0.543100    22    -19.789841  0.013245 -19.776597 -19.748663  1.104588e-02   
0.761700    23    -19.788162  0.021112 -19.767050 -19.746495  1.832327e-02   
1.068500    24    -19.784312  0.031050 -19.753262 -19.742383  2.834166e-02   
1.498800    25    -19.779067  0.041075 -19.737992 -19.736891  3.980669e-02   
2.102400    26    -19.772754  0.049909 -19.722845 -19.730298  5.125064e-02   
2.949000    27    -19.766103  0.056451 -19.709652 -19.723225  6.057272e-02   
4.136600    28    -19.759425  0.060891 -19.698533 -19.715938  6.633822e-02   
5.802500    29    -19.752798  0.064087 -19.688711 -19.708525  6.831717e-02   
8.139200    30    -19.746339  0.066981 -19.679358 -19.701064  6.791154e-02   
11.417000   31    -19.740267  0.070460 -19.669807 -19.693762  6.774828e-02   
16.014700   32    -19.735045  0.075237 -19.659808 -19.687023  7.054855e-02   
22.464000   33    -19.731608  0.081757 -19.649851 -19.681798  7.768778e-02   
31.510500   34    -19.731656  0.090142 -19.641515 -19.680005  8.909951e-02   
44.200100   35    -19.737816  0.099931 -19.637885 -19.685055  1.032809e-01   
62.000000   36    -19.760614  0.106951 -19.653663 -19.709326  1.149682e-01   
85.775000   37    -19.820988  0.108053 -19.712934 -19.776799  1.198441e-01   
109.550000  38    -19.914285  0.109898 -19.804387 -19.881372  1.237246e-01   
133.325000  39    -20.034083  0.125839 -19.908243 -20.013749  1.407478e-01   
157.100000  40    -20.170185  0.161691 -20.008495 -20.162762  1.778053e-01   
180.875000  41    -20.320933  0.212628 -20.108305 -20.326326  2.309999e-01   
204.650000  42    -20.484915  0.277762 -20.207153 -20.502707  2.989741e-01   
228.425000  43    -20.660832  0.356791 -20.304040 -20.690725  3.807786e-01   
252.200000  44    -20.847476  0.449565 -20.397911 -20.888775  4.757180e-01   
275.975000  45    -21.043725  0.555971 -20.487754 -21.095810  5.831956e-01   
299.750000  46    -21.248522  0.675908 -20.572614 -21.310741  7.028209e-01   
323.525000  47    -21.460870  0.809264 -20.651605 -21.532147  8.342268e-01   
347.300000  48    -21.679817  0.955922 -20.723895 -21.759576  9.772357e-01   
371.075000  49    -21.904451  1.115750 -20.7887

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')